In [57]:
# Tentativa de Scraping no Site QuintoAndar
# Camila Junqueira
# Gabriel Nasser
# Rafaela Alexandre

import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

In [4]:
def getDadosCEP(cep):
		url = (f'http://www.viacep.com.br/ws/{cep}/json')
		
		req = requests.get(url)
		if req.status_code == 200:
			dados_json = json.loads(req.text)
			return dados_json
		else:
			print('Erro ao buscar CEP')

In [17]:
meu_cep = "04078012"
dados = getDadosCEP(meu_cep)
bairro = dados['bairro']
dados

{'cep': '04078-012',
 'logradouro': 'Avenida Divino Salvador',
 'complemento': 'de 501/502 a 799/800',
 'unidade': '',
 'bairro': 'Planalto Paulista',
 'localidade': 'São Paulo',
 'uf': 'SP',
 'estado': 'São Paulo',
 'regiao': 'Sudeste',
 'ibge': '3550308',
 'gia': '1004',
 'ddd': '11',
 'siafi': '7107'}

In [18]:
bairro = bairro.replace(" ", "-").lower()
bairro

'planalto-paulista'

In [5]:
qa = "https://www.quintoandar.com.br/comprar/imovel/{bairro}-sao-paulo-sp-brasil"

In [110]:
def scraper(bairro):
    url = f"https://www.quintoandar.com.br/comprar/imovel/{bairro}-sao-paulo-sp-brasil"
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    soup = soup.find('html')
    soup = soup.prettify()
    return soup

In [111]:
moema = scraper(bairro)

In [112]:
soup = BeautifulSoup(moema, 'html.parser')
teste = soup.find_all("div", class_="_2pfuCF")

In [113]:
valor = teste[0].find("h3", class_="EqjlRj").text
valor = re.sub(r'\D', '', valor)
valor

'800000'

In [114]:
m2 = teste[0].find("h3", class_="o6Ojuq").text
m2.split("m²")[0].split(" ")[-2]


'131'

In [104]:
def find_preco_imovel(imovel):
    valor = imovel.find("h3", class_="EqjlRj").text
    return re.sub(r'\D', '', valor)

In [105]:
find_preco_imovel(teste[0])

'800000'

In [106]:
def find_m2_imovel(imovel):
    alt = imovel.find("h3", class_="o6Ojuq").text
    return alt.split("m²")[0].split(" ")[-2]
    

In [107]:
find_m2_imovel(teste[0])

'131'

In [115]:
for i in range(0, 10):
    print("Preco: " + find_preco_imovel(teste[i]))
    print("m2: " + find_m2_imovel(teste[i]))
    print("_"*10)

Preco: 800000
m2: 131
__________
Preco: 387000
m2: 50
__________
Preco: 800000
m2: 120
__________
Preco: 2000000
m2: 90
__________
Preco: 799000
m2: 120
__________
Preco: 610000
m2: 95
__________
Preco: 1590000
m2: 304
__________
Preco: 780000
m2: 140
__________
Preco: 460000
m2: 96
__________
Preco: 480000
m2: 64
__________
